Попытки из odt-файла с субтитрами к сериалу (на русском и немецком) сделать аннотацию ELAN в формате eaf.

In [0]:
from IPython.display import clear_output

In [0]:
!pip install pycodestyle flake8 pycodestyle_magic
!pip install odfpy
clear_output()

In [0]:
%load_ext pycodestyle_magic

In [0]:
!wget https://www.dropbox.com/s/qemuton5skzkmh8/%20annotation.odt?dl=0 -O annotation.odt
clear_output()

In [0]:
import re
import pandas as pd
from string import punctuation
from google.colab import files
from odf.opendocument import load
from odf.table import Table, TableRow, TableCell
from odf import text
from warnings import filterwarnings
filterwarnings("ignore")
clear_output()

# Препроцессинг таблицы odt

In [0]:
annotation = load("annotation.odt")

In [0]:
table = annotation.text.getElementsByType(Table)[0]

In [0]:
df_dict = {"Zeit": [], "Szene": [], "Russisch": [],
           "Deutsch": [], "Bemerkungen": []}
for row in table.getElementsByType(TableRow):
    if str(row) != "ZeitSzeneRussischDeutschBemerkungen" \
            and not str(row).startswith("Gruppe"):
        df_dict["Zeit"].append(str(row.getElementsByType(TableCell)[0]))
        df_dict["Szene"].append(str(row.getElementsByType(TableCell)[1]))
        df_dict["Russisch"].append(str(row.getElementsByType(TableCell)[2]))
        df_dict["Deutsch"].append(str(row.getElementsByType(TableCell)[3]))
        df_dict["Bemerkungen"].append(str(row.getElementsByType(TableCell)[4]))
df = pd.DataFrame(df_dict)

In [9]:
df.head() # не отображаются переносы строки

,Zeit,Szene,Russisch,Deutsch,Bemerkungen
0,0:00,Интро,"«Домашний арест»Сколько денег нужно, чтобы ста...","„Hausarrest“Wie viel Geld braucht man, um glüc...",
1,1:32,Сцена в комуналке,Диктор 1: Говорит и показывает Москва.Диктор 2...,Sprecher 1: Govorit i pokazyvaet Moskva.Sprech...,"Transliteration der ersten zwei Sätze, weil:Fe..."
2,3:21,Сцена в комнате Аркадия,"Иван?: Да спокуха, тут всем хватит.Отец Ивана:...","Szene im Zimmer von ArkadiiIwan: Beruhig dich,...",Der Fernsehsprecher vor dem Dialog fehltKackbr...
3,4:50,В коридоре и на кухне и в туалете,"Женщина в с тазиком в коридоре: Аркашка, скоти...","Im Flur, in der Küche und im BadFrau mit Schüs...",“Pfoten weg” in Ordnung?НЗ это неприкосновенны...
4,6:28,В зале с избирателями,Аркадий Борисович: Спасибо. Спасибо. Как вы вс...,Arkadii Borisowitsch: Danke. Danke. Wie ihr al...,


In [10]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [0]:
# восстанавливаем переносы строки,
# добавляем один перенос в начале каждой строки для удобства
punct = r"([a-zа-яA-ZА-Я\d\"öÖäÄüÜёЁß!»\.“\]\)\?”][!\"#$%&'*+,\-.\/:;<=>?@\\^_`|~»“\)\]\}…”]|\.{3})([A-ZА-ЯЁÄÖÜ])"


def insert_spaces(string):
    return "\n" + re.sub(punct, r"\1\n\2", string)

In [0]:
# это отвратительный костыль, но регуляркой немецкие тексты НЕ ДЕЛЯТСЯ
df[df.columns[1:]] = df[df.columns[1:]].applymap(insert_spaces)
df["Deutsch"][2] = df["Deutsch"][2][:28] + "\n" + df["Deutsch"][2][28:]
df["Deutsch"][3] = df["Deutsch"][3][:33] + "\n" + df["Deutsch"][3][33:]
df["Deutsch"][4] = df["Deutsch"][4].replace(
    " Journalist:", "\nJournalist:")
df["Deutsch"][8] = df["Deutsch"][8].replace(
    "Arkadii Borisowitsch: Hahaha", "Arkadii Borisowitsch: Hahaha\n")
df["Deutsch"][8] = df["Deutsch"][8].replace(
    " Iwan Samsonow:", "\nIwan Samsonow:")
df["Deutsch"][8] = df["Deutsch"][8].replace(
    " Arkadii Borisowitsch:", "\nArkadii Borisowitsch:")
df["Deutsch"][8] = df["Deutsch"][8].replace(
    "Arkadii Borisowitsch Super!", "\nArkadii Borisowitsch: Super!")
df["Deutsch"][8] = df["Deutsch"][8].replace(
    " Pawel Jurjewitsch:", "\nPawel Jurjewitsch:")
df["Deutsch"][8] = df["Deutsch"][8].replace(
    "gesagtPawel Jurjewitsch:", "gesagt\nPawel Jurjewitsch:")
df["Deutsch"][9] = df["Deutsch"][9].replace(
    " Iwan Samsonow:", "\nIwan Samsonow:")
df["Deutsch"][10] = df["Deutsch"][10].replace(
    "Nicht richtig. Nicht in Ordnung. ", "Nicht richtig. Nicht in Ordnung.\n")
df["Deutsch"][10] = df["Deutsch"][10].replace(
    " Iwan Samsonow:", "\nIwan Samsonow:")
df["Deutsch"][14] = df["Deutsch"][14].replace(
    "Nachbarin: Ja", "Nachbarin: Ja\n")
df["Deutsch"][14] = df["Deutsch"][14].replace(
    " Nachbarin:", "\nNachbarin:")
df["Deutsch"][14] = df["Deutsch"][14].replace(
    " Arkadii Borisowitsch:", "\nArkadii Borisowitsch:")
df["Deutsch"][14] = df["Deutsch"][14].replace(
    " Viktoria:", "\nViktoria:")
df["Deutsch"][14] = df["Deutsch"][14].replace(
    " Nina:", "\nNina:")
df["Deutsch"][15] = df["Deutsch"][15].replace(
    " Arkadii Borisowitsch:", "\nArkadii Borisowitsch:")
df["Deutsch"][15] = df["Deutsch"][15].replace(
    " Pawel Ju.:", "\nPawel Ju.:")
df["Deutsch"][15] = df["Deutsch"][15].replace(
    " Iwan Samsonow:", "\nIwan Samsonow:")
df["Deutsch"][15] = df["Deutsch"][15].replace(
    " Stimme des Reporters:", "\nStimme des Reporters:")
df["Deutsch"][15] = df["Deutsch"][15].replace(
    " Pawel Ju. in den Nachrichten:", "\nPawel Ju. in den Nachrichten:")
df["Deutsch"][15] = df["Deutsch"][15].replace(
    "Iwan Samsonow Sag so etwas nicht!", "Iwan Samsonow: Sag so etwas nicht!")
df["Deutsch"][15] = df["Deutsch"][15].replace(
    " Julia:", "\nJulia:")
df["Deutsch"][15] = df["Deutsch"][15].replace(
    " Viktoria:", "\nViktoria:")
df["Deutsch"][15] = df["Deutsch"][15].replace(
    " Nina:", "\nNina:")
df["Deutsch"][15] = df["Deutsch"][15].replace(
    " Marina:", "\nMarina:")

In [0]:
# так как не очень умный автор сценария не знает слова "секретарша",
# пришлось ручками снимать омонимию :)
df["Deutsch"][7] = df["Deutsch"][7].replace("Секретарь:", "Лидия Алексеевна:")

In [0]:
# some minor fixes
df["Russisch"][15] = "\nАркадий: " + df["Russisch"][15][1:]
df["Deutsch"][15] = "\nArkadii: " + df["Deutsch"][15][1:]
df["Russisch"][6] = df["Russisch"][6][:666] + "\n" + df["Russisch"][6][667:]
df["Russisch"][15] = df["Russisch"][15][:6173] + "\n" + df["Russisch"][15][6174:]

In [0]:
# для удобства указываем говорящего в интро и финальных титрах
# убираем переносы строки в монологах рассказчика
df["Russisch"][0] = "Рассказчик: " + df["Russisch"][0][18:]
df["Deutsch"][0] = "Erzähler: " + df["Deutsch"][0][14:]
df["Russisch"][0] = "\n" + df["Russisch"][0].replace("\n", " ")
df["Deutsch"][0] = "\n" + df["Deutsch"][0].replace("\n", " ")
df["Russisch"][15] = df["Russisch"][15].replace(
    "Только я этот счастливый билетик вытащила.",
    "Только я этот счастливый билетик вытащила.\nРассказчик:")
df["Deutsch"][15] = df["Deutsch"][15].replace(
    "Nur ich habe so ein glückliches Los gezogen.\n",
    "Nur ich habe so ein glückliches Los gezogen.\nErzähler: ")
index_ru = df["Russisch"][15].find("Рассказчик: ")
index_de = df["Deutsch"][15].find("Erzähler: ")
df["Russisch"][15] = df["Russisch"][15][:index_ru] + \
                    df["Russisch"][15][index_ru:].replace("\n", " ")
df["Deutsch"][15] = df["Deutsch"][15][:index_de] + \
                    df["Deutsch"][15][index_de:].replace("\n", " ")

In [0]:
# убираем ремарки и заголовки
glitchy_string = "Anonymous2019-05-02T15:45:29.59Ich auch nichtMilen Gavrailov2019-04-27T14:54:01.59Das verstehe ich nicht"
glitchy_string_2 = """Milen Gavrailov2019-05-02T17:31:24.67viel SpaßMilen Gavrailov2019-05-02T17:31:12.67okAnonymous2019-05-02T17:30:55.67Ich wollte bis um acht einfach arbeiten. Mal schauen wie weit ich komme. Der Rest ist dann für dich. :)Milen Gavrailov2019-05-02T17:29:56.67Du musst jetzt nicht auf einmal den ganzen Text übersetzen :DMilen Gavrailov2019-05-02T17:24:20.68даAnonymous2019-05-02T17:21:58.68Dann vielleicht: Das ist doch wieder eine Falle.?
Milen Gavrailov2019-05-02T17:20:12.71разводнякразводняксущ., кол-во синонимов: 8• жульничество (42)• кидалово (19)• лохотрон (19)• надувательство (35)• обман (113)• подстава (25)• развод (31)• разводилово (20)
Anonymous2019-05-02T17:15:35.71Weiß nicht genau, was es bedeutet."""
df["Russisch"][1] = df["Russisch"][1].replace(
    "(Hier bin ich mir nicht sicher)", "")
df["Deutsch"][2] = df["Deutsch"][2][28:]
df["Deutsch"][3] = df["Deutsch"][3][33:]
df["Russisch"][5] = df["Russisch"][5].replace("(6 раз)", "")
df["Deutsch"][5] = df["Deutsch"][5].replace("(6x)", "")
df["Deutsch"][9] = df["Deutsch"][9].replace(" [zu den Kindern]", "")
df["Deutsch"][15] = df["Deutsch"][15].replace(" (zu A. B.)", "")
df["Deutsch"][15] = df["Deutsch"][15].replace(" (ironisch)", "")
df["Russisch"][13] = df["Russisch"][13].replace(glitchy_string, "")
df["Russisch"][14] = df["Russisch"][14].replace(glitchy_string_2, "")
df["Russisch"][14] = df["Russisch"][14].replace(
    "(Ich bin mir nicht sicher)", "")

In [0]:
df["Deutsch"][15] = df["Deutsch"][15].replace(
    " Nachbarin:", "\nNachbarin:")

In [0]:
# переводим столбец Zeit в списки моментов времени

def to_time(string):
    if len(string) > 13:
        res = []
        for i in range(0, len(string) // 10 * 10, 5):
            res.append(string[i: i + 5])
        res.append(string[len(string) // 10 * 10:])
        return res
    elif "-" not in string:
        return [string]
    else:
        return [s.strip(" ") for s in string.split("-")]

In [0]:
df["Zeit"] = df["Zeit"].apply(to_time)

In [20]:
df.head(16)

,Zeit,Szene,Russisch,Deutsch,Bemerkungen
0,[0:00],\nИнтро,"\nРассказчик: Сколько денег нужно, чтобы стать...","\nErzähler: Wie viel Geld braucht man, um glüc...",\n
1,[1:32],\nСцена в комуналке,\nДиктор 1: Говорит и показывает Москва.\nДикт...,\nSprecher 1: Govorit i pokazyvaet Moskva.\nSp...,"\nTransliteration der ersten zwei Sätze, weil:..."
2,[3:21],\nСцена в комнате Аркадия,"\nИван?: Да спокуха, тут всем хватит.\nОтец Ив...","\nIwan: Beruhig dich, es ist genug für alle da...",\nDer Fernsehsprecher vor dem Dialog fehltKack...
3,[4:50],\nВ коридоре и на кухне и в туалете,"\nЖенщина в с тазиком в коридоре: Аркашка, ско...","\nFrau mit Schüssel im Flur: Arkadii, du Dreck...",\n“Pfoten weg” in Ordnung?\nНЗ это неприкоснов...
4,[6:28],\nВ зале с избирателями,\nАркадий Борисович: Спасибо. Спасибо. Как вы ...,\nArkadii Borisowitsch: Danke. Danke. Wie ihr ...,\n
5,[8:02],\nСцена на улице,\nТолпа: Мэр - вандал! \nАркадий Борисович: Ст...,\nMenge: Bürgermeister – Barbar! \nArkadii Bor...,\nEvtl. zu lang.(zu Pawel Ju.)(von Bylinkina a...
6,[11:05],\nСцена в машине,"\nАркадий Борисович: Виктор Степанович, привет...","\nArkadii Borisowitsch: Viktor Stepanowitsch, ...",\n(am Telefon zu seiner Affäre mit dem Decknam...
7,[13:00],\nСцена в офисе /1/,"\nСекретарь: Аркадий Борисович, извините, но в...","\nSekretärin: Arkadii Borisowitsch, verzeihen ...",\nAlternativ: allerdings
8,[15:00],\nСцена в офисе /2/,\nИван Самсонов: Тх...\nТак вроде всё нормальн...,\nIwan Samsonow: Hm...\nSo weit alles in Ordnu...,\nSo weit alles in Ordnung. Eigentlich...alles...
9,[23:00],\nСцена в коммуналке (семья Самсоновых),"\nИван Самсонов: Да, Николай Филиппович, поэто...","\nIwan Samsonow: Ja, Nikolai Filippowitsch, de...",\n???


In [0]:
# сохраняем в более удобном формате
df.to_csv("annotation.csv")
files.download("annotation.csv")

# Извлечение списка говорящих

In [0]:
speakers_ru = []
speakers_de = []

In [0]:
for i in range(len(df["Russisch"])):
    speakers = re.findall(r"\n *([а-яА-ЯёЁ\.\? \d]+?):", df["Russisch"][i])
    for speaker in speakers:
        if speaker not in speakers_ru:
            speakers_ru.append(speaker)

In [0]:
for i in range(len(df["Deutsch"])):
    speakers = re.findall(r"\n *([a-zA-ZäÄöÖüÜ\.\? \d]+?):", df["Deutsch"][i])
    for speaker in speakers:
        if speaker not in speakers_de:
            speakers_de.append(speaker)

In [0]:
# убираем дубликаты
aliases_ru = {"Мужчина в коридоре": "Отец Ивана", "Иван Самсонов": "Иван",
              "Детский голос": "Подросток", "Отец": "Отец Ивана",
              "Сосед": "Петрович",
              "Отец Ивана открывает чемодан": "Отец Ивана", "Иван?": "Иван",
              "Аркадий Борисович": "Аркадий", "Былинкина": "Марина",
              "Студенты": "Толпа", "Жена": "Виктория", "Юля": "Юлия",
              "Марина Сергеевна": "Марина", "Голос": "Капитан",
              "Голос за кадром": "Рассказчик",
              "Голос за дверью": "Человек в форме 1", "Иван Самонов": "Иван",
              "Аркадий Борисович в новостном сюжете": "Аркадий",
              "Голос репортёра": "Репортёр",
              "Павел Юрьевич в новостном сюжете": "Павел Юрьевич"}
speakers_ru = [i for i in speakers_ru if i not in aliases_ru.keys()]

In [0]:
# убираем дубликаты
aliases_de = {"Mann im Flur": "Iwans Vater", "Iwan Samsonow": "Iwan",
              "Kindesstimme": "Teenager", "Vater": "Iwans Vater",
              "Nachbar": "Petrowitsch",
              "Iwans Vater öffnet den Koffer": "Iwans Vater",
              "Wanjas Vater": "Iwans Vater", "Iwan Samsonov": "Iwan",
              "Arkadii Borisowitsch": "Arkadii", "Bylinkina": "Marina",
              "Bylinikina": "Marina", "Studenten": "Menge",
              "Pawel Ju.": "Pawel Jurjewitsch", "Ehefrau": "Viktoria",
              "Sekretärin": "Lidija Aleksejewna", "Iwan S.": "Iwan",
              "Arakelyan": "Arakeljan", "Studentin": "Student",
              "Marina Sergejewna": "Marina", "Stimme": "Kapitän",
              "Voice over": "Erzähler",
              "Stimme hinter der Tür": "Mann in Uniform 1",
              "Mann in Uniform1": "Mann in Uniform 1",
              "Mann in Uniform2": "Mann in Uniform 2",
              "Arkadii Borisowitsch in den Nachrichten": "Arkadii",
              "Die Stimme des Reporters": "Reporter",
              "Stimme des Reporters": "Reporter",
              "Pawel Ju. in den Nachrichten": "Pawel Jurjewitsch"}
speakers_de = [i for i in speakers_de if i not in aliases_de.keys()]

In [27]:
print(len(speakers_ru))
print(len(speakers_de))

34
34


In [0]:
# выравниваем, чтобы каждый говорящий имел один и тот же индекс
# как в speakers_ru, так и в speakers_de
speakers_de[7: 11] = speakers_de[8: 12]
speakers_de[11] = "Iwan"
speakers_de[12], speakers_de[13] = speakers_de[13], speakers_de[12]
speakers_de[15], speakers_de[16] = speakers_de[16], speakers_de[15]
speakers_de[17] = "Sekretär"
speakers_de[18] = "Nina"
speakers_de[24: 28] = speakers_de[25: 29]
speakers_de[28] = "Nachbarin"
speakers_de[30] = "Mann in Uniform 2"
speakers_de[31] = "Viktoria"
speakers_de[32] = "Julia"

In [0]:
# сортируем список говорящих в алфавитном (в русском языке) порядке
speakers = [[ru, de] for ru, de in sorted(zip(speakers_ru, speakers_de))]

In [0]:
# список говорящих тоже сохраняем как датафрейм pandas
df_speakers = pd.DataFrame(speakers, columns=["speakers_ru", "speakers_de"])

In [31]:
df_speakers.head(34)

,speakers_ru,speakers_de
0,Анжела,Angela
1,Аракелян,Arakeljan
2,Аркадий,Arkadii
3,Бабушка Аркадия,Oma von Arkadii
4,Виктория,Viktoria
5,Диктор 1,Sprecher 1
6,Диктор 2,Sprecher 2
7,Женщина в с тазиком в коридоре,Frau mit Schüssel im Flur
8,Журналист,Journalistin
9,Журналистка,Journalist


In [0]:
df_speakers.to_csv("speakers.csv")
files.download("speakers.csv")

# Запись слоёв аннотации в csv-файл

In [0]:
elan_dict = {"Annotation": [], "Tier": [], "Begin time": [],
           "End time": [], "Duration": []}

In [0]:
# "склеиваем" оказавшиеся на разных линиях реплики


def glue(lines, lang):
    if lang == "ru":
        names = speakers_ru + list(aliases_ru.keys())
    elif lang == "de":
        names = speakers_de + list(aliases_de.keys())
    res = []
    for line in lines:
        line = line.strip(" \n")
        if ":" in line and line[:line.index(":")] in names:
            res.append(line)
        elif len(res) > 0:
            res[-1] += line
    return res

In [36]:
# сначала сверяемся, что и в русской, и в немецкой аннотации
# одинаковое количество реплик
for i in range(len(df)):
    lines_ru = glue(df["Russisch"][i].split("\n"), "ru")
    lines_de = glue(df["Deutsch"][i].split("\n"), "de")
    # в переводе две реплики объединили в одну
    # для удобства разбиваем их обратно
    if i == len(df) - 1:
        lines_de = lines_de[:46] + \
            ["Iwan Samsonow: Sie müssen noch die Sachen rausholen."] + \
            ["Iwan Samsonow: Hey! Hallo?!"] + lines_de[47:]
    # то же самое
    if i == 8:
        lines_ru = lines_ru[:23] + \
        ["Аркадий Борисович: Прям так и скажи. Давай набирай. Набирай, набирай. Давай, давай."] + \
        lines_ru[26:]
        lines_de = lines_de[:23] + \
        ["Arkadii Borisowitsch: Ja, genau so. Los, ruf an! Ruf an, ruf an! Los, los!"] + \
        lines_de[25:]
    print(f"lines_ru: {len(lines_ru)}, lines_de: {len(lines_de)}")
# тексты разбиты на реплики корректно

lines_ru: 1, lines_de: 1
lines_ru: 37, lines_de: 37
lines_ru: 5, lines_de: 5
lines_ru: 19, lines_de: 19
lines_ru: 7, lines_de: 7
lines_ru: 27, lines_de: 27
lines_ru: 13, lines_de: 13
lines_ru: 10, lines_de: 10
lines_ru: 71, lines_de: 71
lines_ru: 25, lines_de: 25
lines_ru: 29, lines_de: 29
lines_ru: 7, lines_de: 7
lines_ru: 28, lines_de: 28
lines_ru: 11, lines_de: 11
lines_ru: 138, lines_de: 138
lines_ru: 115, lines_de: 115


In [0]:
# записываем реплики в слои аннотации
for i in range(len(df)):
    lines_ru = glue(df["Russisch"][i].split("\n"), "ru")
    lines_de = glue(df["Deutsch"][i].split("\n"), "de")
    if i == len(df) - 1:
        lines_de = lines_de[:46] + \
            ["Iwan Samsonow: Sie müssen noch die Sachen rausholen."] + \
            ["Iwan Samsonow: Hey! Hallo?!"] + lines_de[47:]
    if i == 8:
        lines_ru = lines_ru[:23] + \
        ["Аркадий Борисович: Прям так и скажи. Давай набирай. Набирай, набирай. Давай, давай."] + \
        lines_ru[26:]
        lines_de = lines_de[:23] + \
        ["Arkadii Borisowitsch: Ja, genau so. Los, ruf an! Ruf an, ruf an! Los, los!"] + \
        lines_de[25:]
    for j in range(len(lines_ru)):
        start = None
        end = None
        dur = None
        name = lines_ru[j][:lines_ru[j].index(":")]
        if name not in speakers_ru:
            name = aliases_ru[name]
        tier_ru = name + "-Spch"
        tier_de = speakers_de[speakers_ru.index(name)] + "-Spch"
        text_ru = lines_ru[j][lines_ru[j].index(":"):].strip(": \n")
        text_de = lines_de[j][lines_de[j].index(":"):].strip(": \n")
        if j == 0:
            start = df["Zeit"][i][0]
        if j == len(lines_ru) - 1 and len(df["Zeit"][i]) > 1:
            end = df["Zeit"][i][0]
        if i == 15:
            if j == 10:
                end = df["Zeit"][i][1]
            elif j == 11:
                start = df["Zeit"][i][1]
            elif j == 21:
                end = df["Zeit"][i][2]
            elif j == 22:
                start = df["Zeit"][i][2]
            elif j == 31:
                end = df["Zeit"][i][3]
            elif j == 32:
                start = df["Zeit"][i][3]
            elif j == 52:
                end = df["Zeit"][i][4]
            elif j == 53:
                start = df["Zeit"][i][4]
            elif j == 63:
                end = df["Zeit"][i][5]
            elif j == 64:
                start = df["Zeit"][i][5]
            elif j == 76:
                end = df["Zeit"][i][6]
            elif j == 77:
                start = df["Zeit"][i][6]
            elif j == 84:
                end = df["Zeit"][i][7]
            elif j == 85:
                start = df["Zeit"][i][7]
            elif j == 91:
                end = df["Zeit"][i][8]
            elif j == 92:
                start = df["Zeit"][i][8]
        elan_dict["Annotation"].append(text_ru)
        elan_dict["Annotation"].append(text_de)
        elan_dict["Tier"].append(tier_ru)
        elan_dict["Tier"].append(tier_de)
        elan_dict["Begin time"].append(start)
        elan_dict["End time"].append(end)
        elan_dict["Duration"].append(dur)
        elan_dict["Begin time"].append(start)
        elan_dict["End time"].append(end)
        elan_dict["Duration"].append(dur)

In [0]:
elan_df = pd.DataFrame(elan_dict)

In [39]:
elan_df.head()

,Annotation,Tier,Begin time,End time,Duration
0,"Сколько денег нужно, чтобы стать счастливым? С...",Рассказчик-Spch,0:00,None,None
1,"Wie viel Geld braucht man, um glücklich zu sei...",Erzähler-Spch,0:00,None,None
2,Говорит и показывает Москва.,Диктор 1-Spch,1:32,None,None
3,Govorit i pokazyvaet Moskva.,Sprecher 1-Spch,1:32,None,None
4,"""Крылатые качели"". Солистка - Оля Новикова.",Диктор 2-Spch,None,None,None


In [0]:
elan_df.to_csv("Übersetzung_1_1.csv")
files.download("Übersetzung_1_1.csv")